In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset("yelp_review_full")

Found cached dataset yelp_review_full (C:/Users/modaj/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/650 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [6]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

C:\Users\modaj\OneDrive\Documents\Personal\Uni\NLP\project\test-venv\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\modaj\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceCl

In [7]:
# from transformers import TrainingArguments

# training_args = TrainingArguments(output_dir="test_trainer")

In [8]:
import evaluate

In [9]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\modaj\OneDrive\Documents\Personal\Uni\NLP\project\test-venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375
  Number of trainable parameters = 108314117


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

## `-`

In [13]:
import subprocess
import psycopg2
import pandas as pd

In [2]:
result = subprocess.run(['heroku', 'pg:credentials:url', 'DATABASE',
                         '--name', 'default', '-a', 'disch'], stdout=subprocess.PIPE, shell = True)
cred_str = result.stdout.split(b'\n')[2].decode("utf-8").strip().strip('"')
cred_str_list = cred_str.split(" ")
creds = {}
for cred in cred_str_list:
    cred_name, cred_val = cred.split('=')
    creds[cred_name] = cred_val

creds

{'dbname': 'd1mikus7g5uss8',
 'host': 'ec2-99-81-68-240.eu-west-1.compute.amazonaws.com',
 'port': '5432',
 'user': 'iohznziolcottb',
 'password': '5a812ea29f6142328bc2afab03e48e6462939babe87610342cdf12e2d357a4f0',
 'sslmode': 'require'}

In [3]:
conn = psycopg2.connect(host=creds['host'], database=creds['dbname'],
                                user=creds['user'],
                                password=creds['password'])
cur = conn.cursor()

In [18]:
select_Query = 'select * from public."DiscH_prototype_question"'
cur.execute(select_Query)
questions = cur.fetchall()
question_col = ["question_id", "description", "num_response", "label_in_question"]
questions_df = pd.DataFrame(questions, columns=question_col)

select_Query = 'select * from public."DiscH_prototype_answer"'
cur.execute(select_Query)
answers = cur.fetchall()
answer_col = ["answer_id", "answer_category_num", "answer_justification", "answer_upvote",
              "account_id_id", "question_id_id", "date"]
answers_df = pd.DataFrame(answers, columns=answer_col)

select_Query = 'select * from public."DiscH_prototype_answer_bow"'
cur.execute(select_Query)
answers_bow = cur.fetchall()
answer_bow_col = ["answer_id", "answer_category_num", "answer_justification", "answer_upvote",
              "account_id_id", "question_id_id", "answer_text_comment", "date"]
answers_bow_df = pd.DataFrame(answers_bow, columns=answer_bow_col)


select_Query = 'select * from public."DiscH_prototype_achievement"'
cur.execute(select_Query)
achievements = cur.fetchall()
achievement_col = ["achievement_id", "achievement_type", "achievement_date", "account_id", "question_id_id", "value"]
achievements_df = pd.DataFrame(achievements, columns=achievement_col)

In [30]:
select_Query = 'select q.description, a.answer_category_num, a.answer_justification, a.answer_upvote, \
              a.account_id_id from public."DiscH_prototype_question" q RIGHT JOIN public."DiscH_prototype_answer" a \
ON a.question_id_id=q.question_id'
cur.execute(select_Query)
QAs = cur.fetchall() 
QA_col = ["question_description", "answer_category_num", "answer_justification", "answer_upvote", "account_id"]
QA_df = pd.DataFrame(QAs, columns=QA_col)

In [32]:
QA_df[["question_description", "answer_category_num"]]

,question_description,answer_category_num
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
1,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
2,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Religious affiliation
3,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,Racist
4,سورية بلد الحضارات تربطها بعلية او بحيوان,Violent
...,...,...
2946,انو انتي عم تعملي جو تأكدي يا اعلامية مين ورا ...,Racist
2947,أقترح ردم اللاجئين بالبحر على طريقة ردم جبل ال...,Violent
2948,اذا كان الكثير من العرب يتهمون الشعب الفلسطيني...,Normal
2949,اجواء اوروبيه بواقع صومالي زنجي حقير,Racist


In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df).train_test_split(test_size=0.2)